In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import normal as normal
from scipy.integrate import odeint
from numpy.linalg import inv

In [2]:
%matplotlib qt

In [3]:
αd = 0.015
η = 60
α0 = 0.03
Ks = 0.05
Ku = 0.0254
K = 80
Kfold = 6000
Pt = 2e6

In [4]:
def heat_shock_response(state, t):
    dt = state[0]
    st = state[1]
    uf = state[2]
    Kd = state[3]
    αs = state[4]
    
    dtd = Kd*st*(1+Ku*uf)/(1+Ku*uf+Ks*dt) - αd*dt
    std = η - α0*st - αs*Ks*dt*st/(1+Ku*uf+Ks*dt)
    ufd = K*(Pt - uf) - (K+Kfold)*dt
    Kdd = 0
    αsd = 0
    
    return np.array([dtd, std, ufd, Kdd, αsd])

In [5]:
dt_original = np.loadtxt('dt_original.txt')
st_original = np.loadtxt('st_original.txt')
dt_noisy = np.loadtxt('dt_noisy.txt')
st_noisy = np.loadtxt('st_noisy.txt')

In [6]:
sampling_times = np.array([10,11,12,15,16,20,40])
sampling_times = np.append(sampling_times, np.arange(50,401,25))

In [7]:
σ1 = (1.24*10**5)**0.5
σ2 = 737.94**0.5
R = np.diag([σ1**2, σ2**2])

### HEKF

In [8]:
def find_f_jacobian(state):
    dt = state[0]
    st = state[1]
    uf = state[2]
    Kd = state[3]
    αs = state[4]
    
    jac = np.zeros((5,5))
    
    jac[0,0] = -αd -Kd*st*(1+Ku*uf)*Ks/(1+Ku*uf+Ks*dt)**2 # df1/dDt
    jac[0,1] = Kd*(1+Ku*uf)/(1+Ku*uf+Ks*dt) # df1/dSt
    jac[0,2] = Ku*Ks*Kd*dt*st/(1+Ku*uf+Ks*dt)**2 # df1/dUf
    jac[0,3] = st*(1+Ku*uf)/(1+Ku*uf+Ks*dt) # df1/dKd
    jac[0,4] = 0 # df1/dαs
    
    jac[1,0] = -(1+Ku*uf)*αs*Ks*st/(1+Ku*uf+Ks*dt)**2 # df2/dDt
    jac[1,1] = -α0 - αs*Ks*dt/(1+Ku*uf+Ks*dt) # df2/dSt
    jac[1,2] = αs*Ks*dt*st*Ku/(1+Ku*uf+Ks*dt)**2 # df2/dUf
    jac[1,3] = 0 # df2/dKd
    jac[1,4] = -Ks*dt*st/(1+Ku*uf+Ks*dt) # df2/dαs
    
    jac[2,0] = -K - Kfold
    jac[2,1] = 0
    jac[2,2] = -K
    jac[2,3] = 0
    jac[2,4] = 0
    
    # $̇Kd = 0, ̇αs = 0$
    
    return jac

In [9]:
def find_h_jacobian():
    jac = np.zeros((2,5)) # only Dt and St measurments are available
    jac[0,0] = 1
    jac[1,1] = 1
    
    return jac

In [10]:
def diff_error_cov_eqn(error_cov_flat, t, jac_f, Q):
    error_cov = np.reshape(error_cov_flat, [5,5])
    error_cov_diff = jac_f@error_cov + error_cov@jac_f.T + Q
    return error_cov_diff.flatten()

In [11]:
states = np.zeros((1+len(sampling_times),5))

# Initial Conditions
state_post = np.array([10000, 50, 2e6, 0.01, 0.01])
Q = 0.1*np.eye(5)
error_cov_post = Q

states[0,:] = state_post

for i in range(len(sampling_times)):
    if i==0:
        t = np.linspace(0, sampling_times[0], 100) 
    else:
        t = np.linspace(sampling_times[i-1], sampling_times[i], 100)
    
    # finding a priori state estimate
    state_prior = odeint(heat_shock_response, state_post, t)[-1,:]
        
    # finding a priori error covariance estimate
    jac_f = find_f_jacobian(state_post)
    error_cov_prior = np.reshape(odeint(diff_error_cov_eqn, error_cov_post.flatten(), 
                                        t, args = (jac_f,Q,))[-1,:], [5,5])

    # finding the gain
    jac_h = find_h_jacobian() #2x5
    gain = error_cov_prior@jac_h.T@inv(jac_h@error_cov_prior@jac_h.T + R) # 5x2

    # finding a posteriori state estimate
    innv = np.reshape(np.array([dt_noisy[i], st_noisy[i]])-state_prior[:2], [2,1])
    state_post = state_prior + np.reshape(gain@innv, [1,5])
    state_post = state_post.flatten()

    states[i+1,:] = state_post

    # finding a posteriori error covariance estimate
    temp = np.eye(5,5) - gain@jac_h
    error_cov_post = temp@error_cov_prior@temp.T + gain@R@gain.T

In [12]:
stimes = np.append([0], sampling_times)

In [56]:
plt.plot(dt_original, label='original')
plt.plot(stimes, states[:,0],'r-', label='estimated')
plt.plot(sampling_times, dt_noisy, '*', label='observed datapoints', mew=5, ms=6)
plt.xlabel('Time')
plt.ylabel('Dt')
plt.title("Number of Chaperones vs. Time")
plt.legend()
plt.savefig("dt.png")

In [57]:
plt.plot(st_original, label='original')
plt.plot(stimes, states[:,1],'r-', label='estimated')
plt.plot(sampling_times, st_noisy, '*', label='observed datapoints', mew=5, ms=6)
plt.xlabel('Time')
plt.ylabel('St')
plt.title("Number of $σ^{32}$ vs. Time")
plt.legend()
plt.savefig("st.png")

In [34]:
plt.title("$K_{d}$ vs. Time")
plt.plot(stimes, states[:,3], 'rx', label='Estimated')
plt.plot(stimes, [3 for i in range(len(stimes))], 'b', label='True Value')
plt.plot(stimes[-14:], states[-14:,3], 'g', label='Steady State')
plt.xlabel('Time')
plt.ylabel('$K_{d}$')
plt.legend()
plt.savefig("kd.png")

In [62]:
plt.title("$α_{s}$ vs. Time")
plt.plot(stimes, states[:,4], 'rx', label='Estimated')
plt.plot(stimes, [3 for i in range(len(stimes))], 'b', label='True Value')
plt.plot(stimes[-14:], states[-14:,4], 'g', label='Steady State')
plt.xlabel('Time')
plt.ylabel('$α_{s}$')
plt.legend()
plt.savefig("alphas.png")

In [63]:
Kd_estimated = np.mean(states[-14:,3])
Kd_estimated

3.025737957438999

In [64]:
αs_estimated = np.mean(states[-14:,4])
αs_estimated

3.046225006761973

### A Posteriori Identifiability Test

In [45]:
t = np.arange(0,401,1)
state0 = [10000, 50, 2e6, Kd_estimated, αs_estimated]
states = odeint(heat_shock_response, state0, t)

In [46]:
noise1_estimates = dt_noisy - states[sampling_times,0].flatten()
noise2_estimates = st_noisy - states[sampling_times,1].flatten()

#### Point Estimates 

In [47]:
var1_estimate = np.dot(noise1_estimates, noise1_estimates)/(len(sampling_times)-1)
var2_estimate = np.dot(noise2_estimates, noise2_estimates)/(len(sampling_times)-1)

In [49]:
print("original variance1: ", σ1**2, " estimated variance1: ", var1_estimate)
print("original variance2: ", σ2**2, " estimated variance2: ", var2_estimate)

original variance1:  124000.00000000001  estimated variance1:  76465.03871924298
original variance2:  737.94  estimated variance2:  961.6030029862185


#### Interval Estimates

Confidence cofficient, $\gamma = 0.95$

$\chi_{22,0.025} = 10.982,~ \chi_{22,0.975} = 36.781$

In [50]:
n12 = np.dot(noise1_estimates, noise1_estimates)
l1 = n12/36.781
u1 = n12/10.982
print("σ1 lies in interval (%d, %d) with 95%% confidence"%(l1,u1))

σ1 lies in interval (43657, 146217) with 95% confidence


In [51]:
n22 = np.dot(noise2_estimates, noise2_estimates)
l2 = n22/36.781
u2 = n22/10.982
print("σ2 lies in interval (%d, %d) with 95%% confidence"%(l2,u2))

σ2 lies in interval (549, 1838) with 95% confidence


### Model 2 

In [13]:
dt_original_m2 = np.loadtxt("dt_original_m2.txt")
st_original_m2 = np.loadtxt("st_original_m2.txt")

In [14]:
αd = 0.015
η = 390
α0 = 0.03
Ks = 0.05
Ku = 0.0254
K = 80
Kfold = 6000
Pt = 2e6
# Kd = 3, need to be estimated
# αs = 3, need to be estimated

In [15]:
def heat_shock_response2(state, t):
    dt = state[0]
    st = state[1]
    uf = state[2]
    Kd = state[3]
    αs = state[4]
    
    dtd = Kd*st*(1+Ku*uf)/(1+Ku*uf+Ks*dt) - αd*dt
    std = η - α0*st - αs*st
    ufd = K*(Pt - uf) - (K+Kfold)*dt
    Kdd = 0
    αsd = 0
    
    return np.array([dtd, std, ufd, Kdd, αsd])

In [16]:
def find_f2_jacobian(state):
    dt = state[0]
    st = state[1]
    uf = state[2]
    Kd = state[3]
    αs = state[4]
    
    jac = np.zeros((5,5))
    
    jac[0,0] = -αd -Kd*st*(1+Ku*uf)*Ks/(1+Ku*uf+Ks*dt)**2 # df1/dDt
    jac[0,1] = Kd*(1+Ku*uf)/(1+Ku*uf+Ks*dt) # df1/dSt
    jac[0,2] = Ku*Ks*Kd*dt*st/(1+Ku*uf+Ks*dt)**2 # df1/dUf
    jac[0,3] = st*(1+Ku*uf)/(1+Ku*uf+Ks*dt) # df1/dKd
    jac[0,4] = 0 # df1/dαs
    
    jac[1,0] = 0 # df2/dDt
    jac[1,1] = -α0 - αs # df2/dSt
    jac[1,2] = 0 # df2/dUf
    jac[1,3] = 0 # df2/dKd
    jac[1,4] = -st # df2/dαs
    
    jac[2,0] = -K - Kfold
    jac[2,1] = 0
    jac[2,2] = -K
    jac[2,3] = 0
    jac[2,4] = 0
    
    # $̇Kd = 0, ̇αs = 0$
    
    return jac

In [17]:
def find_h2_jacobian():
    jac = np.zeros((2,5)) # only Dt and St measurments are available
    jac[0,0] = 1
    jac[1,1] = 1
    
    return jac

In [18]:
def diff_error_cov_eqn(error_cov_flat, t, jac_f, Q):
    error_cov = np.reshape(error_cov_flat, [5,5])
    error_cov_diff = jac_f@error_cov + error_cov@jac_f.T + Q
    return error_cov_diff.flatten()

In [19]:
states2 = np.zeros((1+len(sampling_times),5))

# Initial Conditions
state_post = np.array([10000, 50, 2e6, 0.01, 0.01])
Q = 0.1*np.eye(5)
error_cov_post = Q

states2[0,:] = state_post

for i in range(len(sampling_times)):
    if i==0:
        t = np.linspace(0, sampling_times[0], 100) 
    else:
        t = np.linspace(sampling_times[i-1], sampling_times[i], 100)
    
    # finding a priori state estimate
    state_prior = odeint(heat_shock_response2, state_post, t)[-1,:]
        
    # finding a priori error covariance estimate
    jac_f = find_f2_jacobian(state_post)
    error_cov_prior = np.reshape(odeint(diff_error_cov_eqn, error_cov_post.flatten(), 
                                        t, args = (jac_f,Q,))[-1,:], [5,5])

    # finding the gain
    jac_h = find_h2_jacobian() #2x5
    gain = error_cov_prior@jac_h.T@inv(jac_h@error_cov_prior@jac_h.T + R) # 5x2

    # finding a posteriori state estimate
    innv = np.reshape(np.array([dt_noisy[i], st_noisy[i]])-state_prior[:2], [2,1])
    state_post = state_prior + np.reshape(gain@innv, [1,5])
    state_post = state_post.flatten()

    states2[i+1,:] = state_post

    # finding a posteriori error covariance estimate
    temp = np.eye(5,5) - gain@jac_h
    error_cov_post = temp@error_cov_prior@temp.T + gain@R@gain.T

In [20]:
stimes_200 = stimes[np.where(stimes<=200)]

In [26]:
plt.plot(dt_original[:201], 'yo', label='$\Sigma_{1}$ Ideal Solution')
plt.plot(dt_original_m2[:201], 'co', label='$\Sigma_{2}$ Ideal Solution')
plt.plot(stimes_200, states[:15,0],'r-', label='$\Sigma_{1}$ Estimated Solution')
plt.plot(stimes_200, states2[:15,0],'b-', label='$\Sigma_{2}$ Estimated Solution')
plt.plot(stimes_200, dt_noisy[:15], 'g*', label='observed datapoints', mew=5, ms=6)
plt.title("Number of Chaperones vs. Time")
plt.xlabel('Time')
plt.ylabel('Dt')
plt.legend()
plt.savefig("dt-models.png")

In [27]:
plt.plot(st_original[:201], 'yo', label='$\Sigma_{1}$ Ideal Solution')
plt.plot(st_original_m2[:201], 'co', label='$\Sigma_{2}$ Ideal Solution')
plt.plot(stimes_200, states[:15,1],'r-', label='$\Sigma_{1}$ Estimated Solution')
plt.plot(stimes_200, states2[:15,1],'b-', label='$\Sigma_{2}$ Estimated Solution')
plt.plot(stimes_200, st_noisy[:15], 'g*', label='observed datapoints', mew=5, ms=6)
plt.title("Number of $σ^{32}$ vs. Time")
plt.xlabel('Time')
plt.ylabel('St')
plt.legend()
plt.savefig("st-models.png")

In [33]:
plt.title("$K_{d}$-Model2 vs. Time")
plt.plot(stimes, states2[:,3], 'rx', label='Estimated')
plt.plot(stimes, [3 for i in range(len(stimes))], 'b', label='True Value')
plt.plot(stimes[-13:], states2[-13:,3], 'g', label='Steady State')
plt.xlabel('Time')
plt.ylabel('$K_{d}$')
plt.legend()
plt.savefig("kd-m2.png")

In [35]:
plt.title("$α_{s}$-Model2 vs. Time")
plt.plot(stimes, states2[:,4], 'rx', label='Estimated')
plt.plot(stimes, [3 for i in range(len(stimes))], 'b', label='True Value')
plt.plot(stimes[-13:], states2[-13:,4], 'g', label='Steady State')
plt.xlabel('Time')
plt.ylabel('$α_{s}$')
plt.legend()
plt.savefig("alphas-m2.png")

In [36]:
Kd_estimated = np.mean(states2[-13:,3])
Kd_estimated

3.0307649890523463

In [37]:
αs_estimated = np.mean(states2[-13:,4])
αs_estimated

2.983211003825827

In [38]:
t = np.arange(0,401,1)
state0 = [10000, 50, 2e6, Kd_estimated, αs_estimated]
states3 = odeint(heat_shock_response2, state0, t)

In [39]:
noise1_estimates = dt_noisy - states3[sampling_times,0].flatten()
noise2_estimates = st_noisy - states3[sampling_times,1].flatten()

In [40]:
var1_estimate = np.dot(noise1_estimates, noise1_estimates)/(len(sampling_times)-1)
var2_estimate = np.dot(noise2_estimates, noise2_estimates)/(len(sampling_times)-1)

In [41]:
print("original variance1: ", σ1**2, " estimated variance1: ", var1_estimate)
print("original variance2: ", σ2**2, " estimated variance1: ", var2_estimate)

original variance1:  124000.00000000001  estimated variance1:  13302612.34081991
original variance2:  737.94  estimated variance1:  12117.181186351067


In [43]:
n12 = np.dot(noise1_estimates, noise1_estimates)
l1 = n12/36.781
u1 = n12/10.982
print("var1 lies in interval (%d, %d) with 95%% confidence"%(l1,u1))

var1 lies in interval (7595086, 25437521) with 95% confidence


In [44]:
n22 = np.dot(noise2_estimates, noise2_estimates)
l2 = n22/36.781
u2 = n22/10.982
print("var2 lies in interval (%d, %d) with 95%% confidence"%(l2,u2))

var2 lies in interval (6918, 23170) with 95% confidence
